# A Simple Workflow with Prefect

## Install required Python packages

In [ ]:
!pip install prefect numpy

## Import packages

In [2]:
import random
import numpy as np
from time import sleep

from prefect import task, flow

## Define a set of functions as tasks

Adding the @task decorator is all it takes.

In [3]:
@task
def get_number(min=1, max=100) -> int:
    """Return a random number n, wit min<=n<max"""
    sleep(1) # simulate longer processing
    n = random.randint(min,max)
    print (f"random number: {n}")
    return n

In [4]:
@task
def add(n1:int, n2: int) -> int:
    """Return sum of two numbers"""
    sleep(1.5) # simulate longer processing
    s = n1 + n2
    print (f"sum: {s}")
    return s

In [5]:
@task
def multiply(n1: int, n2: int) -> int:
    """Return product of two numbers"""
    sleep(1) # simulate longer processing
    p = n1 * n2
    print (f"product: {p}")
    return p

In [6]:
@task
def mean(*args: int) -> float:
    """Return the mean of prvided numbers"""
    sleep(3) # simulate longer processing
    m = np.mean(args)
    print (f"mean: {m}")
    return m

## Define the data pipeline as a flow

Adding the @flow decorator does the trick. In addition:
- We're also giving our flow a name, "my pipeline",
- redirecting all print statements to be collected as Prefect logs,
- and we `submit` each function for concurrent execution.

In [7]:
@flow(name="concurrent analysis", log_prints=True)
def analysis():
    """Return results of some arbitrary calculations"""
    n1 = get_number.submit()
    n2 = get_number.submit()
    temp_sum = add.submit(n1, n2)
    n3 = get_number.submit()
    r = multiply.submit(n3, temp_sum)
    m = mean.submit(n1,n2,n3)
    return r, m

# Run the data pipeline 

Local execution is triggered by invoking the function with the @flow decorator.

In the output you will see the logs of the Prefect execution engine, including a URL to the Prefect Server that observes flow and provides visualization of all task dependencies, execution states and logs.

In [8]:
r, m = analysis()
print (f"arbitrary calc: r={r}, mean={m}")

16:24:13.690 | INFO    | Flow run 'able-magpie' - Beginning flow run 'able-magpie' for flow 'concurrent analysis'

16:24:13.701 | INFO    | Flow run 'able-magpie' - View at https://sds-prefect.pods.uvarc.io/runs/flow-run/54c93089-809c-4c79-8b42-7d1a942881d2

16:24:14.864 | INFO    | Task run 'get_number-447' - random number: 61

16:24:14.867 | INFO    | Task run 'get_number-391' - random number: 70

16:24:14.868 | INFO    | Task run 'get_number-b04' - random number: 74

...

16:24:17.434 | INFO    | Task run 'multiply-c36' - Finished in state Completed()

16:24:17.900 | INFO    | Task run 'mean-b0a' - mean: 68.33333333333333

16:24:17.909 | INFO    | Task run 'mean-b0a' - Finished in state Completed()

16:24:17.949 | INFO    | Flow run 'able-magpie' - Finished in state Completed('All states completed.')

arbitrary calc: r=Completed(), mean=Completed()
